# 2. 신경망 기본 구성 요소

__감사말__

프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 2장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전 이상에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

__구글 코랩 설정__

'런타임 -> 런타임 유형 변경' 메뉴에서 GPU를 지정한다.
TensorFlow 버전을 확인하려면 아래 명령문을 실행한다.

In [3]:
import tensorflow as tf
tf.__version__

'2.17.0'

TensorFlow가 GPU를 사용하는지 여부를 확인하려면 아래 명령문을 실행한다.
아래와 같은 결과가 나오면 GPU가 제대로 지원됨을 의미한다.

```
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
```

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

**참고**

아래 코드에 대한 자세한 설명은
[신경망 구성 요소](https://codingalzi.github.io/dlp2/building_blocks_of_NN.html)를
참고한다.

## 신경망 모델 기초 훈련법

**케라스로 MNIST 데이터셋 불러오기**

In [5]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

훈련셋 정보

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
len(train_labels)

60000

In [8]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

테스트셋 정보

In [9]:
test_images.shape

(10000, 28, 28)

In [10]:
len(test_labels)

10000

In [11]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

**신경망 모델의 구조 지정**

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

**신경망 모델 컴파일**

In [14]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

**이미지 데이터 전처리**

In [15]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255   # 0과 1사이의 값
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255     # 0과 1사이의 값

**검증셋 지정**

In [ ]:
val_images = train_images[:10000]    # 검증셋. 샘플 1만 개
train_images = train_images[10000:]  # 훈련셋. 샘플 5만 개

val_labels = train_labels[:10000]    # 검증셋 타깃
train_labels = train_labels[10000:]  # 훈련셋 타깃

**모델 훈련**

In [17]:
model.fit(train_images, 
          train_labels, 
          epochs=5, 
          batch_size=128,
          validation_data=(val_images, val_labels))

Epoch 1/5


I0000 00:00:1727006967.269417    2702 service.cc:146] XLA service 0x7f0ec4006ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727006967.269449    2702 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9


146/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8015 - loss: 0.6790

I0000 00:00:1727006968.666584    2702 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8639 - loss: 0.4668 - val_accuracy: 0.9565 - val_loss: 0.1548
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.9621 - loss: 0.1293 - val_accuracy: 0.9684 - val_loss: 0.1072
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9763 - loss: 0.0831 - val_accuracy: 0.9720 - val_loss: 0.0933
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9822 - loss: 0.0575 - val_accuracy: 0.9746 - val_loss: 0.0822
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9882 - loss: 0.0390 - val_accuracy: 0.9772 - val_loss: 0.0742


## 훈련된 모델 활용과 평가

**모델 활용: 예측하기**

In [18]:
test_digits = test_images[0:10]
predictions = model.predict(test_digits)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step


0번 이미지에 대한 예측값

In [19]:
predictions[0]

array([1.0921791e-07, 2.5011984e-09, 5.5916903e-06, 1.5574026e-04,
       7.9578705e-11, 1.3535265e-07, 1.7049874e-11, 9.9983382e-01,
       1.3471417e-07, 4.4301414e-06], dtype=float32)

가장 높은 확률값을 갖는 인덱스는 7.

In [20]:
predictions[0].argmax()

7

0번 이미지가 가리키는 숫자가 7일 확률이 99.98% 정도.

In [21]:
predictions[0][7]

0.9998338

실제로 첫째 이미지의 레이블이 7임.

In [22]:
test_labels[0]

7

**모델 성능 테스트**

In [23]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step - accuracy: 0.9745 - loss: 0.0823
test_acc: 0.9775000214576721
